# Hawkes fit to start times
Load the imports:

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.stats as st
from matplotlib.ticker import FuncFormatter
import scipy.optimize as op
plt.style.use("dark_background") # Config plots for dark mode, delete if on light mode
plt.rcParams['figure.dpi'] = 150 # Hi-res plots

Load the station data:

In [2]:
station_data = pd.read_csv("../data/santander_locations.csv")
station_data.head() # Load the station data and inspect the first 5 rows
class StationIdError(IndexError):
    """Called when we try and read a non-existing station Id"""
    pass

def get_station_name(in_id):
    """Get station name from bike_data for a given id, catching any exceptions"""
    try:
        return station_data[station_data["Station.Id"] == in_id].StationName.iloc[0]
    except IndexError:
        StationIdError("No station matching input ID")

Preprocess the start times to make them pseudo-continuous:

In [54]:
bike_data = pd.read_csv("../data/processed_df.csv", index_col=0)
bike_data.head() # Load the processed bike data and inspect the first 5 rows

# Find minimum start time
x = bike_data.min()["start_time"]
t_min = (x // 86400) * 86400

# Substract t_min from start_time and end_time
bike_data["start_time"] = (bike_data["start_time"] - t_min) / 60
bike_data["end_time"] = (bike_data["end_time"] - t_min) / 60

# Introduce random perturbations to make pseudo-continuous
bike_data["start_time"] = bike_data["start_time"] + np.random.rand(*bike_data["start_time"].shape)
bike_data["end_time"] = bike_data["end_time"] + np.random.rand(*bike_data["end_time"].shape)

bike_data["duration"] = bike_data.end_time - bike_data.start_time
bike_data = bike_data.sort_values(by=["start_time"])

train_time = 12*7*24*60
train_bike_data = bike_data[bike_data.start_time <= train_time]
train_bike_data.head()

train_sorted_stations = []
for st_id in train_bike_data.start_id.sort_values().unique():
    train_sorted_stations.append(train_bike_data[train_bike_data.start_id==st_id])

train_sorted_stations[0].head()

rates_dict = {}
for station in train_sorted_stations:
    time_elapsed = station.start_time.to_numpy()[-1] - station.start_time.to_numpy()[0]
    n_events = train_sorted_stations[0].size
    rate = n_events / time_elapsed

    rates_dict[station.start_id.unique()[0]]= rate


We will use the following kernel:
$$
\mu_i(t) = \alpha_i e^{-\beta_i t},\, 0 \leq \alpha_i \leq \beta_i
$$

In [58]:
def genA(i, beta, t):
    if i == 0:
        return 0
    else:
        return np.exp(-1*beta*(t[i] - t[i-1]))*(1+genA(i-1, beta, t))

def hawkes_log_likelihood(t, alpha, beta, lambda_p): 
    A = []
    for i in range(0, len(t)):
        if i==0:
            A.append(0)
        else:
            A.append(np.exp(-1*beta*(t[i] - t[i-1]))*(1+A[i-1]))
    l = 0
    for i in range(0, len(t)):
        l += np.log(lambda_p + alpha*A[i]) + (alpha/beta) * (np.exp(-beta*(t[-1] - t[i])) - 1)
    l -= lambda_p * t[-1]
    return l

hawkes_log_likelihood(train_sorted_stations[0].start_time.to_numpy(),1,1,1)


-122435.850871605

In [38]:
# log(alpha) = log(beta + alpha)
# op_hawkes_likelihood = lambda x: -hawkes_log_likelihood(station.start_time.to_numpy(), np.exp(x[0]), np.exp(x[1]), np.exp(x[2]) + np.exp(x[1]))
# x0 = [1,2,10]
# To constrain, take exp, and to revert take log

In [61]:
class OptimizationError(RuntimeError):
    pass

optimal_parameters = {}
for station in train_sorted_stations:
    print(station.start_id.unique()[0])
    x0 = [rates_dict[station.start_id.unique()[0]],2,10]
    op_hawkes_likelihood = lambda x: -hawkes_log_likelihood(station.start_time.to_numpy(), np.exp(x[0]), np.exp(x[1]), np.exp(x[2]) + np.exp(x[1]))
    sol = op.minimize(op_hawkes_likelihood, x0)
    if sol.success:
        max_params = sol.x
        optimal_parameters[station.start_id.unique()[0]] = max_params
    else:
        raise OptimizationError("Optimizer does not converge")
        break
optimal_parameters

1


/var/folders/82/t8qzqs6x2f19402hdk543xd80000gn/T/ipykernel_22680/3045833632.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  l += np.log(lambda_p + alpha*A[i]) + (alpha/beta) * (np.exp(-beta*(t[-1] - t[i])) - 1)
/var/folders/82/t8qzqs6x2f19402hdk543xd80000gn/T/ipykernel_22680/3045833632.py:16: RuntimeWarning: invalid value encountered in double_scalars
  l += np.log(lambda_p + alpha*A[i]) + (alpha/beta) * (np.exp(-beta*(t[-1] - t[i])) - 1)
/var/folders/82/t8qzqs6x2f19402hdk543xd80000gn/T/ipykernel_22680/3045833632.py:16: RuntimeWarning: divide by zero encountered in log
  l += np.log(lambda_p + alpha*A[i]) + (alpha/beta) * (np.exp(-beta*(t[-1] - t[i])) - 1)
/var/folders/82/t8qzqs6x2f19402hdk543xd80000gn/T/ipykernel_22680/3045833632.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  l += np.log(lambda_p + alpha*A[i]) + (alpha/beta) * (np.exp(-beta*(t[-1] - t[i])) - 1)
/var/folders/82/t8qzqs6x2f19402hdk543xd80000gn/T/ipykernel_22680/3045833632.py:1

OptimizationError: Optimizer does not converge

In [100]:
def compensator(t, alpha, beta, lambda_p):
    comp = []
    for k in range(len(t)):
        app = 0
        app += t[k]*lambda_p
        for i in range(0,k+1):
            app += (alpha/beta) * np.exp(-beta * (t[k] - t[i]))

def goodcompensator(t, alpha, beta, lambda_p):
    result = []
    for tk in t:
        app = lambda_p*tk - 



In [77]:
stations = list(optimal_parameters.keys())
alphas = [item[0] for item in optimal_parameters.values()]
betas = [item[1] for item in optimal_parameters.values()]
lambdas = [item[2] for item in optimal_parameters.values()]
#hawkes_parameters = pd.DataFrame((optimal_parameters.values()), columns = ["alpha", "beta", "lambda" ], index=optimal_parameters.keys())
#hawkes_parameters.to_csv('../data/hawkes_parameters.csv')

In [103]:
hawkes_parameters = pd.read_csv('../data/hawkes_parameters.csv', index_col=0)
zeroth_case = hawkes_parameters.to_numpy()[0]
zeroth_time = train_sorted_stations[0].start_time.to_numpy()
compensated_times = compensator(zeroth_time, zeroth_case[0], zeroth_case[1], zeroth_case[2])
compensated_times

404.5913763657923